In [ ]:
# attempt to setup and run the model as close to the author's description as possible
# he uses a 66/33 train/test split and evaluates performance of 25 runs, each with different random samples

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
main_data = pd.read_csv("./data/train.csv")

# 'critical_temp' is the target
X = main_data.drop('critical_temp', axis=1)
y = main_data['critical_temp']


# Create a baseline XGBoost model with the parameters specified in the paper
xgb_model = XGBRegressor(
    n_estimators=374,         # Tree size: 374
    max_depth=16,             # Maximum depth: 16
    learning_rate=0.02,       # Learning rate (η): 0.02
    min_child_weight=1,       # Minimum child weight: 1
    colsample_bytree=0.5,     # Column subsampling: 0.50
    random_state=42,
    objective='reg:squarederror'  # default: reg:squarederror
)


# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 66/33 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42 + i)
    
    # Fit the model on the training set
    xgb_model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = xgb_model.predict(X_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")




Results of reconstructed original model with original features and original 66/33 split:

Run 1: RMSE = 9.4656, R² = 0.9230
Run 2: RMSE = 9.5316, R² = 0.9223
Run 3: RMSE = 9.4369, R² = 0.9241
Run 4: RMSE = 9.4620, R² = 0.9233
Run 5: RMSE = 9.6280, R² = 0.9205
Run 6: RMSE = 9.6689, R² = 0.9211
Run 7: RMSE = 9.6060, R² = 0.9211
Run 8: RMSE = 9.5098, R² = 0.9218
Run 9: RMSE = 9.3343, R² = 0.9257
Run 10: RMSE = 9.6704, R² = 0.9209
Run 11: RMSE = 9.5213, R² = 0.9227
Run 12: RMSE = 9.8162, R² = 0.9198
Run 13: RMSE = 9.3990, R² = 0.9237
Run 14: RMSE = 9.2016, R² = 0.9280
Run 15: RMSE = 9.3278, R² = 0.9260
Run 16: RMSE = 9.5248, R² = 0.9229
Run 17: RMSE = 9.3086, R² = 0.9261
Run 18: RMSE = 9.6788, R² = 0.9205
Run 19: RMSE = 9.4905, R² = 0.9236
Run 20: RMSE = 9.3468, R² = 0.9258
Run 21: RMSE = 9.5593, R² = 0.9229
Run 22: RMSE = 9.7150, R² = 0.9194
Run 23: RMSE = 9.5224, R² = 0.9219
Run 24: RMSE = 9.6434, R² = 0.9208
Run 25: RMSE = 9.5375, R² = 0.9222

Average RMSE over 25 runs: 9.5163

Average R² over 25 runs: 0.9228



In [1]:
# run the author's original model 
# with a 90/10 train/test split and evaluates performance of 25 runs, each with different random samples

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
main_data = pd.read_csv("./data/train.csv")

# 'critical_temp' is the target
X = main_data.drop('critical_temp', axis=1)
y = main_data['critical_temp']


# Create a baseline XGBoost model with the parameters specified in the paper
xgb_model = XGBRegressor(
    n_estimators=374,         # Tree size: 374
    max_depth=16,             # Maximum depth: 16
    learning_rate=0.02,       # Learning rate (η): 0.02
    min_child_weight=1,       # Minimum child weight: 1
    colsample_bytree=0.5,     # Column subsampling: 0.50
    random_state=42,
    objective='reg:squarederror' # default: reg:squarederror
)


# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 90/10 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42 + i)
    
    # Fit the model on the training set
    xgb_model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = xgb_model.predict(X_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")




Run 1: RMSE = 8.5532, R² = 0.9362
Run 2: RMSE = 9.3544, R² = 0.9236
Run 3: RMSE = 9.0177, R² = 0.9293
Run 4: RMSE = 8.8060, R² = 0.9331
Run 5: RMSE = 9.5005, R² = 0.9217
Run 6: RMSE = 9.1928, R² = 0.9250
Run 7: RMSE = 9.1327, R² = 0.9293
Run 8: RMSE = 9.0252, R² = 0.9297
Run 9: RMSE = 8.5435, R² = 0.9382
Run 10: RMSE = 8.5260, R² = 0.9386
Run 11: RMSE = 9.4515, R² = 0.9261
Run 12: RMSE = 10.2299, R² = 0.9138
Run 13: RMSE = 8.9638, R² = 0.9316
Run 14: RMSE = 8.6929, R² = 0.9351
Run 15: RMSE = 8.9946, R² = 0.9330
Run 16: RMSE = 9.5404, R² = 0.9209
Run 17: RMSE = 8.7495, R² = 0.9349
Run 18: RMSE = 8.7018, R² = 0.9355
Run 19: RMSE = 9.6553, R² = 0.9187
Run 20: RMSE = 8.8650, R² = 0.9315
Run 21: RMSE = 9.2875, R² = 0.9268
Run 22: RMSE = 9.8380, R² = 0.9177
Run 23: RMSE = 8.9450, R² = 0.9324
Run 24: RMSE = 9.6714, R² = 0.9215
Run 25: RMSE = 8.6743, R² = 0.9357

Average RMSE over 25 runs: 9.1165
Average R² over 25 runs: 0.9288


Results of reconstructed original model with original features and original 90/10 split:

Run 1: RMSE = 8.5532, R² = 0.9362
Run 2: RMSE = 9.3544, R² = 0.9236
Run 3: RMSE = 9.0177, R² = 0.9293
Run 4: RMSE = 8.8060, R² = 0.9331
Run 5: RMSE = 9.5005, R² = 0.9217
Run 6: RMSE = 9.1928, R² = 0.9250
Run 7: RMSE = 9.1327, R² = 0.9293
Run 8: RMSE = 9.0252, R² = 0.9297
Run 9: RMSE = 8.5435, R² = 0.9382
Run 10: RMSE = 8.5260, R² = 0.9386
Run 11: RMSE = 9.4515, R² = 0.9261
Run 12: RMSE = 10.2299, R² = 0.9138
Run 13: RMSE = 8.9638, R² = 0.9316
Run 14: RMSE = 8.6929, R² = 0.9351
Run 15: RMSE = 8.9946, R² = 0.9330
Run 16: RMSE = 9.5404, R² = 0.9209
Run 17: RMSE = 8.7495, R² = 0.9349
Run 18: RMSE = 8.7018, R² = 0.9355
Run 19: RMSE = 9.6553, R² = 0.9187
Run 20: RMSE = 8.8650, R² = 0.9315
Run 21: RMSE = 9.2875, R² = 0.9268
Run 22: RMSE = 9.8380, R² = 0.9177
Run 23: RMSE = 8.9450, R² = 0.9324
Run 24: RMSE = 9.6714, R² = 0.9215
Run 25: RMSE = 8.6743, R² = 0.9357

Average RMSE over 25 runs: 9.1165
Average R² over 25 runs: 0.9288